## **TODO** - move to xlsx files

## Create FHIRSTU3 SearchParameter Resource, CababilityStatement and Quick start text, and Searchparameter list. Using the Python FHIRClient Module and SearchParameter CSV file with source data.

### Prerequisites:

- Python 3.6 or greater


### Import FHIRClient and other libraries

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from fhirclient.models import searchparameter as SP
from fhirclient.models import capabilitystatement as CS
from fhirclient.models import bundle as B
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython import display as Display
from pprint import pprint
from collections import namedtuple
from pandas import read_csv
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape

#### Inspect ElementProperties as reference

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [ ]:
#******************** Need to update *************************************************
base_id = "argo-q"
canon_base = "http://fhir.org/guides/argonaut-questionnaire/"

publisher = 'The Argonaut Project'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'https://github.com/argonautproject/questionnaire/issues'
                        ) 

#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''
#out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
out_path=''
#out_path = "C:/Users/Eric/Documents/Jan_2019_FHIR_Experience"
#sp_csv = 'SearchParams.csv'
#sp_csv = 'Q_SearchParam.csv'
sp_csv = 'Argonaut__Clinical_Notes_SearchParameters'

#***********************************************************************************

md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

fhir_term_server = 'http://test.fhir.org/r3'
fhir_test_server = 'http://test.fhir.org/r3'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
fhir_base_url = 'http://hl7.org/fhir/'


none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False']
sep_list = (',', ';', ' ', ', ', '; ')
search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]'
    )


### validate

In [ ]:
# *********************** validate Resource ********************************

def validate(r):
    fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r3'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
  
    }
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [ ]:
 def write_file(name, data): # write file
    # out_path = ''
    out_path = '//ERICS-AIR-2/ehaas/Documents/FHIR/Argo-Questionnaire/source/resources/'
    with open(f'{Path(out_path)}/{name}.json', 'w') as f:
        f.write(data)

### Get Search Parameter input data

In [ ]:
data = read_csv(f'{in_path}{sp_csv}')
data

In [ ]:
with open(f'{in_path}{sp_csv}') as f:
    reader = csvreader(f)
    top_row = next(reader)
    top_row = [t.lower().split(' ') for t in top_row]
    top_row = ['_'.join(t) for t in top_row]

    Data = namedtuple("Data", top_row)
    data = [Data(*r) for r in reader]
    
for d in data:
    print(f'Resource = {d.resource}, Search Parameter = {d.parameter}, Exists = {d.exists}')
    

### update core SP with additional capabiliities

#### Get core SP (d.exists == 'Y')

- Get definitions bundle
- If need to update SP Extract the SP based on the csv file

In [ ]:
def_path = 'C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/STU3_Definitions/search-parameters.json'  # use '/' !

print(Path(def_path))

with open(Path(def_path)) as f:
    b = B.Bundle(load(f))

b.id


In [ ]:
def sp_multipleAnd(sp):
    x = X.Extension(dict(
    url = f'{canon_base}multipleAnd',
    valueBoolean = True
    ))
    try:
        sp.extension.append(x)
    except AttributeError:
        sp.extension = [x]
    # print('add adds extension')
    return sp
    
  
def sp_multipleOr(sp):
    x = X.Extension(dict(
    url = f'{canon_base}multipleOr',
    valueBoolean = True
    ))
    try:
        sp.extension.append(x)
    except AttributeError:
        sp.extension = [x]
    # print('add ors extension')
    return sp


new_constraints = {
    'ands': sp_multipleAnd,
    'ors': sp_multipleOr,
    'modifiers': 'modifier',
    'comparators': 'comparator',
    'references': 'target',
    'chains': 'chain'
    }

sp_list=[]
for d in data:
    fullUrl = f'http://hl7.org/fhir/SearchParameter/{d.resource}-{d.parameter}'
    for i in b.entry:
        if i.fullUrl == fullUrl and d.update == 'Y': #update sp
            # print(dumps(i.resource.as_json(),indent = 3))
            sp = i.resource
            # change id and url, publisher, and contact, draft etc
            sp.derivedFrom =sp.url
            #sp.id = f'{base_id}-{sp.id}'
            sp.url = f'{canon_base}/SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = sp.id.replace('-','_').lower()
            sp.status = 'active'

            for k,v in new_constraints.items():
                print(getattr(d,k))
                if getattr(d,k) =='Y':
                    v(sp)
                elif getattr(d,k) not in none_list:
                    a_list = getattr(d,k).split(',')
                    # print(a_list)
                    setattr(sp,v,a_list)



            print(dumps(sp.as_json(),indent = 3))

            #validate and write to file

            print('...validating')
            # print(validate(sp))
            #r = validate(sp)
            #display(Display.HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))


            # save to file

            rjson = dumps(sp.as_json(), indent=3)
            name =f'searchparameter-{sp.id.lower()}'
            print(name)
            # write files as json
            # write_file(name, rjson)
            sp_list.append(sp)
            break

        elif i.fullUrl == fullUrl:
            sp = i.resource
            sp_list.append(sp)
            break


for i in sp_list:
    print(i.id, i.name, i.url)

                             
                 

### Create SP Resources by assigning CSV data to SP elements

sp_list = []



def split_string(s):
    for r in sep_list:
        s = s.replace(r, ' ')
    return s.split()

for d in data:
    #initiliaze
    sp = SP.SearchParameter()
    sp.date = D.FHIRDate(datetime.utcnow().isoformat())
    sp.publisher = publisher
    sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
    # print(d.parameter)
    sp.id = f'{base_id}-{d.resource.lower()}-{d.parameter}'
    sp.url = f'{canon_base}{sp.id}'
    sp.name = sp.id.replace('-','_')
    sp.derivedFrom = f'http://hl7.org/fhir/SearchParameter/{d.resource}-{d.parameter.lower()}' if d.exists == 'Y' else None
    # print(sp.derivedFrom)
    sp.status = 'active'
    sp.description = d.description
    sp.code = d.parameter.lower()
    sp.base = [d.resource]  # list
    sp.type = d.type.lower()
    # print(sp.type)
    sp.expression = d.expression
    sp.target = [] if d.reference in none_list else split_string(d.reference) # string as space separated list in csv
    # print(sp.target)
    # sp.multipleOr = True  add columns to csv for where needed or used otherwise leave blank
    # sp.multipleAnd = True  add columns to csv for where needed or used
    sp.comparator = [] if d.comparators in none_list else split_string(d.comparators) # string as space separated list in csv
    # print(sp.comparator)
    sp.modifier = [] if d.modifiers in none_list else split_string(d.modifiers) # string as space separated list in csv
    # print(sp.modifier)
    sp.chain = [] if d.chains in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.chain)
    # TODO sp.combo = [] if d.combos in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.combo)
    sp.conformance = d.conformance_server.upper()
    
    sp_list.append(sp)
    
    


### Display SP resources

for row,i in enumerate(sp_list):
    print(row,i.base)
    print(dumps(sp_list[1].as_json(), indent = 3))



### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles

In [ ]:
r_type = ['Questionnaire', 'QuestionnaireResponse', 'Patient']  # TODO  loop over all types in sp_list

template = env.get_template(md_template[0])
for t in r_type:
    sp_bytype = [i for i in sp_list if i.base[0] == r_type]
    display(Markdown(template.render(sp_bytype=sp_bytype,search_type=search_type)))
    search_md = template.render(sp_bytype=sp_bytype,search_type=search_type)

with open(f'{out_path}{r_type}-search.md','w') as f:
    f.write(search_md)



### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [ ]:
# get types:
r_list = []
for i in sp_list:
    for j in i.base:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])
display(Markdown(template.render(sp_list=sp_list,r_list=r_list)))
searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)
with open(f'{out_path}searchparameters.md','w') as f:
    f.write(searchparameters_md)

### Update CapabilityStatement for Search
- instantiate new or existing CapStatement as fhir model
- add combination extensions
- add markdown descriptions from above

In [ ]:
# instantiate new or existing CapStatement:

def interaction(code):
    return CS.CapabilityStatementRestResourceInteraction({'code' : code})

def search_cs(sp):
    template = env.get_template(md_template[2])
    return CS.CapabilityStatementRestResourceSearchParam(
    dict(
    name = sp.code,
    definition = sp.url,
    type = sp.type,
    documentation = template.render(sp=sp, search_type=search_type)
    )
    )

cs = CS.CapabilityStatement(dict(
                            date = now,
                            status = 'active',
                            acceptUnknown = 'both',
                            format = ['json','xml'],
                            fhirVersion = '3.0.1',
                            kind = 'requirements'     
                            ))



cs.rest = [CS.CapabilityStatementRest()]  
cs.rest[0].mode = 'server'
cs.rest[0].resource = []



print(r_list)    
for r in r_list:
    rest_resource = CS.CapabilityStatementRestResource()
    rest_resource.type = r
    rest_resource.interaction = [interaction(code) for code in ['read','vread', 'history-instance', 'search-type']]
    cs.rest[0].resource.append(rest_resource)
    rest_resource.searchParam = [search_cs(sp=sp) for sp in sp_list if sp.base[0] == r]
print(dumps(cs.as_json(),indent = 3))
    
    



### TODO

- Save SearchParameters and CapabilityStatements.
- Cycle through the quick starts
- Update the CSV files